In [1]:
import os
import re
import pandas as pd
import numpy as np
import datasets
from datasets import load_dataset, load_metric, Dataset, concatenate_datasets,DatasetDict
from datasets import load_from_disk
from tqdm import tqdm
tqdm.pandas(position=0,leave=True)
import itertools
import spacy
nlp = spacy.load("en_core_web_md")
from textblob import TextBlob
# python -m textblob.download_corpora
import string
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/ec2-user/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [43]:
all_stopwords_gensim = STOPWORDS.union(set(['thank','thanks', 'you', 'help','questions','a.m.','p.m.','friday','thursday','wednesday','tuesday','monday',\
                                            'askunum','email','askunum.com','unum','askunumunum.com','day','use', 'appreciate','available','mailtoaskunumunum.com',\
                                            'hello','hi','online','?','.','. .','phone','needs','need','let','know','service','information','time','meet','client',\
                                           'team','ask','file','date','opportunity','original','benefit','eastern','specialists','specialist','attached','experienced',\
                                            'benefits insurance','employee','click','organization','httpsbit.lycjrbm',  'received', 'billing', 'manager', 'assist', \
                                            'additional', 'response']))

appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "i would",
"i'd" : "i had",
"i'll" : "i will",
"i'm" : "i am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "i have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

phrases = ['caution external email: this email originated from outside of the organization', 
           'do not click links or open attachments unless you recognize the sender and know the content is safe', 
           'this information is for official use only',
           'this message originated outside of unum. use caution when opening attachments, clicking links or responding to requests for information',
           'this email message and its attachments are for the sole use of the intended recipient or recipients and may contain confidential information',
           'if you have received this email in error, please notify the sender and delete this message',
           'this email and any files transmitted with it are confidential and intended solely for the use of the individual or entity to whom they are addressed',
           'unauthorized disclosure or misuse of this personal information including, but not limited to copying, disclosure, distribution, is strictly prohibited, and may result in criminal and/or civil penalties',
           'if you have any questions, we have experienced service specialists available to help you monday through friday',
           'original message',
           'BenefitMall Customer Service',
           'please let us know if there is anything else that we can assist you with',
           'we appreciate the opportunity to meet your benefit needs',
           '8 a.m. to 8 p.m. eastern time',
           'eastern',
           '8 a.m. to 8 p.m.',
           'thank you for contacting ask unum',
           'it was my pleasure to assist you today',
           'i hope my email finds you well',
           'please feel free to let us know if there is anything further we may assist you with',
           'please feel free to contact us with any other questions or requests we may assist you with',
           'please let us know if we can be of further assistance',
           'click here',
           'thank you for your email'
           ]

In [26]:
def text_preprocess(text, extract_adj=False):
    # lemma = nltk.wordnet.WordNetLemmatizer()
    
    text = str(text)
    
    #remove http links from the email
    
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], '')  
    
    text = re.sub("`", "'", text)
    
    #fix misspelled words

    '''Here we are not actually building any complex function to correct the misspelled words but just checking that each character 
    should occur not more than 2 times in every word. It’s a very basic misspelling check.'''

    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))

    
    text = [appos[word] if word in appos else word for word in text.lower().split()]
    text = " ".join(text)
    
    ### Remove stop word
    text = [i for i in word_tokenize(text) if i not in all_stopwords_gensim]
    text = " ".join(text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    #Remove punctuation
    table = str.maketrans('', '', string.punctuation)
    text = [w.translate(table) for w in text.split()]
    text=" ".join(text)
    
    # stem
    # ps = PorterStemmer()
    # text=" ".join(set([ps.stem(w) for w in text.split()]))
    
    if extract_adj:
        ADJ_word=[]
        doc=nlp(text)
        for token in doc:
            if token.pos_=="ADJ":
                ADJ_word.append(token.text)   
        return " ".join(ADJ_word)
    
    else:
        return text
    
def textblob_sentiment(text):
    pol_score = TextBlob(text).sentiment.polarity
    if pol_score > 0: 
        return 'positive'
    elif pol_score == 0: 
        return 'neutral'
    else: 
        return 'negative'

def vader_sentiment(text):
    
    senti = SentimentIntensityAnalyzer()
    compound_score = senti.polarity_scores(text)['compound']
    
    # set sentiment 
    if compound_score >= 0.05: 
        return 'positive'
    elif (compound_score > -0.05) and (compound_score < 0.05): 
        return 'neutral'
    else: 
        return 'negative'

In [12]:
email_all=load_from_disk(os.path.join(os.getcwd(),"dataset","email_all"))
email_all

DatasetDict({
    train: Dataset({
        features: ['Full_TextBody', 'Client_TextBody', 'Latest_TextBody', 'year', 'churn'],
        num_rows: 156414
    })
    test: Dataset({
        features: ['Full_TextBody', 'Client_TextBody', 'Latest_TextBody', 'year', 'churn'],
        num_rows: 27497
    })
})

In [13]:
train_data=email_all['train']
test_data=email_all['test']
train_data.set_format(type="pandas")
df_train=train_data[:]
test_data.set_format(type="pandas")
df_test=test_data[:]

In [14]:
tempt1=pd.DataFrame(df_train["churn"].value_counts(dropna=False)).reset_index().rename(columns={'index':'churn','churn':'count'})
tempt2=pd.DataFrame(df_train["churn"].value_counts(dropna=False,normalize=True)).reset_index().rename(columns={'index':'churn','churn':'percentage'})
tempt1.merge(tempt2, on="churn", how="inner").style.format({'count':'{:,}','percentage':'{:.2%}'}).set_caption("Training set churn dist")\
.set_table_styles([{'selector': 'caption','props': [('color', 'red'),('font-size', '15px')]}])

,churn,count,percentage
0,0,"133,904",85.61%
1,1,"22,510",14.39%


In [15]:
tempt1=pd.DataFrame(df_test["churn"].value_counts(dropna=False)).reset_index().rename(columns={'index':'churn','churn':'count'})
tempt2=pd.DataFrame(df_test["churn"].value_counts(dropna=False,normalize=True)).reset_index().rename(columns={'index':'churn','churn':'percentage'})
tempt1.merge(tempt2, on="churn", how="inner").style.format({'count':'{:,}','percentage':'{:.2%}'}).set_caption("Test set churn dist")\
.set_table_styles([{'selector': 'caption','props': [('color', 'red'),('font-size', '15px')]}])

,churn,count,percentage
0,0,"23,786",86.50%
1,1,"3,711",13.50%


In [48]:
for p in phrases:
    df_test["Full_TextBody"]= df_test["Full_TextBody"].str.replace(p, ' ')
df_test["Full_TextBody"].iloc[1]

"our ask unum team should be able to provide you with the schedule as. and ill send you a census for december 2019 on the employer paid coverage in a separate secured email. please see request below for schedule as on policy 593441 for information and resources regarding unum's covid-19 response and faqs, visit unum.com/covid-19 plan administrators for a comprehensive website demonstration of the administration of unum's voluntary worksite benefits as well as common faqs, help is here 24/7 we appreciate your business and the opportunity to assist you with this request. if you have any questions, we have experienced representatives available to help you monday through friday from   at our toll-free number 1-800-ask-unum 1-800-275-8686 or email us at.thanks for your request. could you please confirm the plan date and the policy numbers which you need these reports for? please let us know if you need anything else. have a great day  . ,  ..on tue, apr 6, 2021 at 128 pm wrote thanks for yo

In [46]:
text="""
"our ask unum team should be able to provide you with the schedule as. and ill send you a census for december 2019 on the employer paid coverage in a separate secured email. 
please see request below for schedule as on policy 593441 for information and resources regarding unum's covid-19 response and faqs, 
visit unum.com/covid-19 plan administrators for a comprehensive website demonstration of the administration of unum's voluntary worksite benefits as well as common faqs, 
help is here 24/7 we appreciate your business and the opportunity to assist you with this request. if you have any questions, 
we have experienced representatives available to help you monday through friday from 8 a.m. to 8 p.m. eastern time at our toll-free number 1-800-ask-unum 1-800-275-8686 or 
email us at.thanks for your request. could you please confirm the plan date and the policy numbers which you need these reports for? please let us know if you need anything else. 
have a great day we appreciate the opportunity to meet your benefit needs. , 8 a.m. to 8 p.m. eastern time..on tue, apr 6, 2021 at 128 pm wrote thanks for your request. 
could you please confirm the plan date and the policy numbers which you need these reports for? please let us know if you need anything else. 
have a great day we appreciate the opportunity to meet your benefit needs. , 8 a.m. to 8 p.m. eastern time..this request was re-opened as they are needing employee census 
information. the schedule a data was provided so i am sending an email to have a request created to the right area for the census information. screen shot below to speak to one of 
our highly qualified customer service representatives, please call 1-800-ask-unum or email them at this message may contain confidential and privileged information. 
if it has been sent to you in error, please reply to advise the sender of the error and then immediately delete this message..thank you for your email. 
we have attached the requested census listings for cvca lead llc. please feel free to contact us with any other questions or requests we may assist you with. 
we appreciate the opportunity to meet your benefit needs. , 8 a.m. to 8 p.m. eastern time."
"""

In [41]:
for p in phrases:
    text= text.replace(p, ' ')

In [42]:
text

'\n"our ask unum team should be able to provide you with the schedule as. and ill send you a census for december 2019 on the employer paid coverage in a separate secured email. \nplease see request below for schedule as on policy 593441 for information and resources regarding unum\'s covid-19 response and faqs, \nvisit unum.com/covid-19 plan administrators for a comprehensive website demonstration of the administration of unum\'s voluntary worksite benefits as well as common faqs, \nhelp is here 24/7 we appreciate your business and the opportunity to assist you with this request. if you have any questions, \nwe have experienced representatives available to help you monday through friday from   at our toll-free number 1-800-ask-unum 1-800-275-8686 or \nemail us at.thanks for your request. could you please confirm the plan date and the policy numbers which you need these reports for? please let us know if you need anything else. \nhave a great day  . ,  ..on tue, apr 6, 2021 at 128 pm wr

### Test Set

In [27]:
for p in phrases:
    df_test["Full_TextBody"]= df_test["Full_TextBody"].replace(p, ' ', regex=False)
df_test["bag_of_word"]=df_test["Full_TextBody"].progress_apply(text_preprocess)

100%|██████████| 27497/27497 [04:54<00:00, 93.49it/s] 


In [ ]:
def extract_adj(text):
    ADJ_word=set()
    doc=nlp(text)
    for token in doc:
        if token.pos_=="ADJ":
            ADJ_word.add(token.text)
    return ADJ_word

adj_count=Counter()
tempt_test=df_test[df_test["churn"]==1]
for index,row in tqdm(tempt_test.iterrows(), total=tempt_test.shape[0]):
    adj_word=extract_adj(row['Full_TextBody'])
    adj_count.update(adj_word)
    adj,freq=zip(*adj_count.most_common(50))
for i ,j in zip(adj,freq):
    print("{:<20}{:<20,}".format(i,j))

In [ ]:
# tempt=df_test.copy()
# tempt["set_word"]=tempt["Full_TextBody"].progress_apply(lambda x: set(x.split()))
# tempt["terminate"]=tempt["set_word"].progress_apply(lambda x: 1 if set(["i'll"]).issubset(x) else 0 )
# tempt[tempt["terminate"]==1]["Full_TextBody"].iloc[2]

In [ ]:
tempt=df_test.copy()
tempt["set_word"]=tempt["Full_TextBody"].progress_apply(lambda x: set(x.split()))
tempt["terminate"]=tempt["set_word"].progress_apply(lambda x: 1 if set(["termination","terminate"]).issubset(x) else 0 )
tempt["terminate"].value_counts()

In [ ]:
tempt[(tempt["terminate"]==1) & (tempt["churn"]==0)]["Full_TextBody"].iloc[2]

In [ ]:
tempt=df_test[df_test["churn"]==1]
tempt.shape

In [ ]:
tempt.head(2)

In [ ]:
tempt["bag_of_word"]=tempt["Full_TextBody"].progress_apply(text_preprocess)
tempt.head(2)

In [ ]:
tempt["adj_bag_of_word"]=tempt["Full_TextBody"].progress_apply(lambda x: text_preprocess(x, extract_adj=True))
tempt.head(2)

In [ ]:
tempt["sentiment"]=tempt["adj_bag_of_word"].progress_apply(textblob_sentiment)
tempt.head(2)

In [ ]:
tempt["vader_sentiment"]=tempt["adj_bag_of_word"].progress_apply(vader_sentiment)
tempt.head(2)

In [ ]:
tempt.sentiment.value_counts()

In [ ]:
tempt.vader_sentiment.value_counts()

In [ ]:
tempt_test=tempt[tempt['vader_sentiment']=="negative"]
neg_word_test=set()
for index,row in tqdm(tempt_test.iterrows(), total=tempt_test.shape[0]):
    neg_word_test=neg_word_test | set(row["adj_bag_of_word"].split())
neg_word_test

In [ ]:
tempt_test["Latest_TextBody"].iloc[16]

In [ ]:
# text='active'
# TextBlob(text).sentiment.polarity

# from textblob import TextBlob

# testimonial = TextBlob("active")
# print(testimonial.sentiment)

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# analyzer = SentimentIntensityAnalyzer()
# sentence = "The food was terrible!" 
# vs = analyzer.polarity_scores(sentence)
# print("{:-<65} {}".format(sentence, str(vs)))

In [ ]:
# !pip install --quiet flair
# from flair.models import TextClassifier
# from flair.data import Sentence

# classifier = TextClassifier.load('en-sentiment')
# sentence = Sentence('The food was great!')
# classifier.predict(sentence)

# # print sentence with predicted labels
# print('Sentence above is: ', sentence.labels)

### Training set

In [ ]:
tempt=df_train[df_train["churn"]==1]
tempt.shape

In [ ]:
tempt["bag_of_word"]=tempt["Full_TextBody"].progress_apply(text_preprocess)
tempt.head(2)

In [ ]:
tempt["adj_bag_of_word"]=tempt["Full_TextBody"].progress_apply(lambda x: text_preprocess(x, extract_adj=True))
tempt.head(2)

In [ ]:
tempt["sentiment"]=tempt["adj_bag_of_word"].progress_apply(textblob_sentiment)
tempt.head(2)

In [ ]:
tempt["vader_sentiment"]=tempt["adj_bag_of_word"].progress_apply(vader_sentiment)
tempt.head(2)

In [ ]:
tempt.sentiment.value_counts()

In [ ]:
tempt.vader_sentiment.value_counts()

In [ ]:
tempt_train=tempt[tempt['vader_sentiment']=="negative"]
tempt_train.head()

In [ ]:
neg_word=set()
for index,row in tqdm(tempt_train.iterrows(), total=tempt_train.shape[0]):
    neg_word=neg_word | set(row["adj_bag_of_word"].split())
neg_word